In [43]:
import os
import re
import requests
import zipfile
from urllib.request import urlopen
from io import BytesIO
import pandas as pd
import geopandas as gp

- access the person file from each year
- calculate total fatalities for each group for each year and add to new dataframe
- calculate relative fatalities for each group for each year and add to new dataframe

- split by race?

In [109]:
# list of years for later use
yr_range = [x for x in range(1975, 2021)]

person_names = [
    "person.csv",
    "person.CSV",
    "PERSON.csv",
    "PERSON.CSV"
]

# create a list of paths to access the FARS FTP site
paths = [
    "https://static.nhtsa.gov/nhtsa/downloads/FARS/" + 
    str(x) + 
    "/National/FARS" + 
    str(x) + 
    "NationalCSV.zip" for x in yr_range
]

# check
paths[10]

'https://static.nhtsa.gov/nhtsa/downloads/FARS/1985/National/FARS1985NationalCSV.zip'

In [110]:
def find_filename(name_list, file_list):
    """
    Output the exact name of a file by checking
    against a predefined list of filenames. Used to
    deal with different filename cases across years
    in FARS data.
    
    - name_list: predefined list of names
    - file_list: list of file names to search through
    """
    filename = "None" # initialize a filename variable
    
    # run through list of files
    for file in file_list:
        if file in name_list: # check if list of file is in predefined list
            filename = file # if so, set filename
    if filename == "None": # if not, raise error
        raise ValueError(
            'There were no matches between the predefined list {} and list {}'.format(name_list, file_list))
        
    return filename

def fetch_multi_csv_zip_from_url(url, filenames=[], *args, **kwargs):
    """
    Modified from:
    https://stackoverflow.com/questions/48843761/how-to-scrape-csv-
    files-from-a-url-when-they-are-saved-in-a-zip-file-in-pytho
    """
    assert kwargs.get('compression') is None
    
    req = urlopen(url)
    
    zip_file = zipfile.ZipFile(BytesIO(req.read()))
    
    names = zip_file.namelist() # get a list of files in the archive
    filename = find_filename(filenames, names) # find the right filename
    
    # check that the specified file names are actually in the zip
    if filename not in names:
        raise ValueError(
            'filename {} not in {}'.format(filename, names))
    
    # read the file into a dataframe
    df = pd.read_csv(zip_file.open(filename), *args, **kwargs)
    return df




# test
url = 'https://static.nhtsa.gov/nhtsa/downloads/FARS/2010/National/FARS2010NationalCSV.zip'

person = fetch_multi_csv_zip_from_url(
    url, 
    filenames=person_names, 
    encoding_errors="surrogateescape"
)

person.head()

STATE  ST_CASE  VE_FORMS  VEH_NO  PER_NO  N_MOT_NO  COUNTY  DAY  MONTH  \
0      1    10001         1       1       1         0      81   15      1   
1      1    10001         1       1       2         0      81   15      1   
2      1    10002         1       1       1         0      35   11      1   
3      1    10003         3       1       1         0      97   14      1   
4      1    10003         3       2       1         0      97   14      1   

   HOUR  ...  LAG_HRS  LAG_MINS  P_SF1  P_SF2  P_SF3       CERT_NO  WORK_INJ  \
0     4  ...        0         0      0      0      0  ************         0   
1     4  ...      999        99      0      0      0  ************         8   
2     6  ...        0        35      0      0      0  ************         0   
3    15  ...        0        46      0      0      0  ************         0   
4    15  ...      999        99      0      0      0  ************         8   

   HISPANIC  RACE LOCATION  
0         7     2        0  
1         0     0        0  
2         7     2        0  
3         7     1        0  
4         0     0        0  

[5 rows x 82 columns]

In [117]:
# next is to start looping through the years of data